In [1]:
import numpy as np
import pandas as pd
import json
from pprint import pprint
from dataclasses import asdict
import matplotlib.pyplot as plt

from martignac.workflows.solute_in_solvent_alchemical import SoluteInSolventAlchemicalFlow
from martignac.nomad.entries import get_entries_of_upload, get_entries_in_database, get_entries_of_my_uploads, get_entry_by_id
from martignac.nomad.uploads import get_specific_file_from_upload, get_upload_by_id, get_all_my_uploads
from martignac.nomad.queries import find_mini_queries_corresponding_to_workflow

from martignac.workflow_interfaces.solute_in_solvent_alchemical import SoluteInSolventAlchemicalInterface
from martignac.workflow_interfaces.utils import convert_entry_id_to_specific_interface

from martignac.nomad.utils import get_nomad_request

24-08-22 10:46:33 - pymbar.timeseries - WARNING - Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
24-08-22 10:46:33 - pymbar.mbar_solvers - WARNING - 
********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************
24-08-22 10:46:33 - pymbar.mbar_solvers - INFO - JAX was either not detected or disabled, using standard NumPy and SciPy
24-08-22 10:46:34 - MDAnalysis.coordinates.AMBER - WARNING - netCDF4 is not available. Writing AMBER ncdf files will be slow.


In [2]:
df = pd.json_normalize([asdict(e) for e in find_mini_queries_corresponding_to_workflow(SoluteInSolventAlchemicalFlow(), dataset_ids=["HJdEI1q4SV-c5Di43BTT_Q"])])
df = df.rename(
    columns={
        "comment.job_id": "job_id", 
        "comment.state_point.lambda_state": "lambda_state",
        "comment.state_point.solute_name": "solute_name", 
        "comment.state_point.solvent_name": "solvent_name", 
        "comment.mdp_files": "mdp_files", 
        "comment.itp_files": "itp_files"
    }
)
df = df.sort_values("lambda_state").groupby(by=["solute_name", "solvent_name"]).first()
df = df.drop(["workflow_name", "comment.workflow_name", "datasets", "comment.state_point.type", "lambda_state"], axis=1)
df["free_energy_in_kt"] = df["entry_id"].apply(lambda x: convert_entry_id_to_specific_interface(x, use_prod=SoluteInSolventAlchemicalFlow.nomad_use_prod_database).free_energy.mean)
df["nomad_url"] = df["entry_id"].apply(lambda x: get_entry_by_id(x, use_prod=SoluteInSolventAlchemicalFlow.nomad_use_prod_database).nomad_gui_url)
column_free_energy = df.pop("free_energy_in_kt")
column_nomad_url = df.pop("nomad_url")
df.insert(0, "free_energy_in_kt", column_free_energy)
df.insert(1, "nomad_url", column_nomad_url)
df

24-08-22 10:46:35 - martignac.nomad.queries - INFO - querying comments on test server
24-08-22 10:46:35 - martignac.nomad.queries - INFO - querying entries with query {'query': {'authors.name': {'any': ['Tristan Bereau', 'Joseph Rudzinski']}, 'datasets.dataset_id': {'any': ('HJdEI1q4SV-c5Di43BTT_Q',)}, 'entry_type': 'Workflow'}, 'pagination': {'page_size': 25}, 'required': {'include': ['entry_id', 'upload_id', 'comment', 'published', 'datasets.*']}}
24-08-22 10:46:35 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 10:46:35 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 10:46:35 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 10:46:36 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 10:46:36 - martignac.

free_energy_in_kt  \
solute_name solvent_name                      
P6          CHEX                  -0.437799   
            CLF                    2.406969   
            ETH                    2.814678   
            HD                    -4.014783   
            W                      7.272538   

                                                                  nomad_url  \
solute_name solvent_name                                                      
P6          CHEX          https://nomad-lab.eu/prod/v1/test/gui/user/upl...   
            CLF           https://nomad-lab.eu/prod/v1/test/gui/user/upl...   
            ETH           https://nomad-lab.eu/prod/v1/test/gui/user/upl...   
            HD            https://nomad-lab.eu/prod/v1/test/gui/user/upl...   
            W             https://nomad-lab.eu/prod/v1/test/gui/user/upl...   

                                              entry_id  \
solute_name solvent_name                                 
P6          CHEX          S4CkHz37dr-6VqqC01pm0D1LMTON   
            CLF           Uh1vpRNbc4LWar87CwOE6_VaQx6I   
            ETH           zHvALbcL2sK7fGLWtbxD9bKFvihf   
            HD            S_hio-NjCCfNra1BKZ8i1aJlMhKy   
            W             l4aFpbvnfzLE4Z1NXsOXGVwzN0FP   

                                       upload_id  published  use_prod  \
solute_name solvent_name                                                
P6          CHEX          A74Kxh-dRx-376A9yuXDIg       True     False   
            CLF           F21IQDA-TOi8hrneDZ9Smg       True     False   
            ETH           Tmb7J-P8ScKe3THBUvsqiw       True     False   
            HD            KxDvzQ6HRVaICMOfSxeLxQ       True     False   
            W             8bpyz_tjSSi0TNp8bvfwkg       True     False   

                                                    job_id  \
solute_name solvent_name                                     
P6          CHEX          e30741699db6e4a0f8c104fc0f7feb59   
            CLF           8926c803cd797330ee7c8e545b713874   
            ETH           a31dbcf33ebf932e65b938e4f74f136d   
            HD            a2ad31a1658772eaefad79a864db51a9   
            W             b3823ab96ff5fb05bffd22fdeb119ec7   

                                                 mdp_files  \
solute_name solvent_name                                     
P6          CHEX          89b51ecb08ca7fb84bfa3118d1ede5b6   
            CLF           89b51ecb08ca7fb84bfa3118d1ede5b6   
            ETH           89b51ecb08ca7fb84bfa3118d1ede5b6   
            HD            89b51ecb08ca7fb84bfa3118d1ede5b6   
            W             89b51ecb08ca7fb84bfa3118d1ede5b6   

                                                 itp_files  
solute_name solvent_name                                    
P6          CHEX          15259a565eb93668e9e40eb8cfd10f9b  
            CLF           15259a565eb93668e9e40eb8cfd10f9b  
            ETH           15259a565eb93668e9e40eb8cfd10f9b  
            HD            15259a565eb93668e9e40eb8cfd10f9b  
            W             15259a565eb93668e9e40eb8cfd10f9b